In [2]:
%load_ext autoreload
%autoreload 2

import lzma
import dill
import json
import pandas as pd
import shap
from tqdm.auto import tqdm
import numpy as np
import tensorflow as tf
from pathlib import Path

import scienceplots

import matplotlib.pyplot as plt
import matplotlib as mpl

from matal import load_model, load_data, auto_log, DATA_DIR, MODEL_DIR, augmente_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
X_COLS = ['MMT', 'CNF', 'GEL', 'GLY']
Y_COLS = ['VLTransmittance', 'UVTransmittance', 'IRTransmittance',
          'AshAreaRatio', 
          'Stress', 'Strain', 'Modulus']

round_df = load_data('14.train_data__da0.table.csv')
round_df

,sID,MMT,CNF,GEL,GLY,VLTransmittance,UVTransmittance,IRTransmittance,AshAreaRatio,Stress,Strain,Modulus,Toughness,CurveAlpha,CurveBeta
0,aGS-0102,0.5766,0.0512,0.2396,0.1326,43.133333,7.966667,69.833333,0.896762,59.778394,0.009711,7157.164972,0.317155,0.707540,0.374980
1,aGS-0103,0.4781,0.1542,0.1472,0.2205,66.866667,16.766667,76.533333,0.681098,43.173864,0.012239,5266.890797,0.325823,0.817078,0.113220
2,aGS-0104,0.0190,0.4228,0.1231,0.4351,81.900000,48.500000,78.533333,0.113607,11.967871,0.293099,637.816764,2.637192,0.656708,0.240143
3,aGS-0105,0.0545,0.6615,0.0946,0.1894,85.300000,59.433333,89.766667,0.208607,52.697660,0.067738,3195.180044,2.613561,0.689250,0.031860
4,aGS-0106,0.4682,0.1940,0.3216,0.0162,56.600000,14.900000,68.133333,0.902425,89.668495,0.029374,4418.278036,1.456433,0.389384,0.081807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,aGS-F10F,0.0444,0.4116,0.0295,0.5145,85.700000,55.866667,85.900000,0.214230,12.531151,0.040783,1381.632020,0.394629,0.783157,0.035202
131,aGS-F110,0.4302,0.2044,0.3444,0.0210,67.666667,39.166667,68.733333,0.935640,69.740532,0.018774,5489.131084,0.759460,0.650269,0.348022
132,aGS-F111,0.4286,0.3530,0.0332,0.1852,75.066667,34.866667,79.500000,0.690702,66.250711,0.024103,5325.954486,1.055530,0.820831,0.362030
133,aGS-F112,0.5998,0.2043,0.1721,0.0238,30.000000,8.333333,59.166667,0.790141,82.753010,0.033130,6110.239149,1.853577,0.774353,0.457703


In [4]:
augmente_data(round_df, n_augmented=1000, x_cols=X_COLS, y_cols=Y_COLS, x_std=0.05, y_std=0.05, random_state=1)

,sID,DA#,MMT,CNF,GEL,GLY,VLTransmittance,UVTransmittance,IRTransmittance,AshAreaRatio,Stress,Strain,Modulus
0,aGS-0102,0,0.576600,0.051200,0.239600,0.132600,43.133333,7.966667,69.833333,0.896762,59.778394,0.009711,7157.164972
1,aGS-0102,1,0.593879,0.092281,0.256122,0.067442,43.134342,8.015849,69.785026,0.934313,59.774032,0.066256,7157.188273
2,aGS-0102,2,0.572482,0.089006,0.267713,0.103115,43.063664,7.969722,69.845384,0.969123,59.816082,-0.019694,7157.195130
3,aGS-0102,3,0.553387,0.103654,0.234058,0.104125,43.219831,7.936989,69.819154,0.906718,59.731653,-0.030395,7157.162094
4,aGS-0102,4,0.602951,0.059806,0.254031,0.164274,43.078582,7.956029,69.788731,0.904360,59.809596,0.039685,7157.198255
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135130,aGS-F113,996,0.220923,0.321380,0.360503,-0.054642,79.318534,53.135980,76.703314,0.684124,64.622481,0.002278,5730.781293
135131,aGS-F113,997,0.347154,0.367717,0.370407,0.108401,79.329599,53.198607,76.740019,0.681936,64.653196,-0.027815,5730.755197
135132,aGS-F113,998,0.249626,0.438840,0.332554,0.014702,79.288321,53.204750,76.815317,0.668299,64.618657,0.028022,5730.837655
135133,aGS-F113,999,0.200768,0.395664,0.294394,0.102907,79.341177,53.244775,76.768343,0.723136,64.628742,-0.024892,5730.718162
